#Evidence Retrieval Module for fetching evidence from Wikipedia

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
mkdir data

In [0]:
cd data

In [0]:
!wget https://s3-eu-west-1.amazonaws.com/fever.public/train.jsonl && wget https://s3-eu-west-1.amazonaws.com/fever.public/shared_task_dev.jsonl && wget https://s3-eu-west-1.amazonaws.com/fever.public/shared_task_test.jsonl

In [0]:
!wget https://www.dropbox.com/s/05on2t6edciczx3/valid.tsv && wget https://www.dropbox.com/s/sp81ovwrovozz53/train.tsv && wget https://www.dropbox.com/s/sggvga937n8a2xx/test.tsv && wget https://www.dropbox.com/s/68m38xy1ws5xgpg/liar_evidence.tsv

In [0]:
!wget https://s3-eu-west-1.amazonaws.com/fever.public/wiki-pages.zip

In [0]:
!unzip wiki-pages.zip 

In [0]:
#!wget https://nlp.stanford.edu/software/stanford-postagger-full-2018-02-27.zip

In [0]:
#!unzip stanford-postagger-full-2018-02-27.zip

In [0]:
!pip install -U gensim

In [0]:
!wget https://github.com/facebookresearch/fastText/archive/v0.1.0.zip


Redirecting output to ‘wget-log.8’.


In [0]:
!unzip v0.1.0.zip

In [0]:
cd fastText-0.1.0

/content/data/fastText-0.1.0


In [0]:
!make

In [0]:
ls

In [0]:
!wget https://codeload.github.com/epfml/sent2vec/zip/master


Redirecting output to ‘wget-log’.


In [0]:
!unzip master

In [0]:
cd sent2vec-master/src

/content/data/fastText-0.1.0/sent2vec-master/src


In [0]:
!pip install cython

In [0]:
!python setup.py build_ext

In [0]:
!pip install .

In [0]:
cd ../../..

/content/data


### Code for getting CPU and GPU information.

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))


In [0]:
# Importing necessary libraries
import numpy as np
import os
from os.path import join, exists, split
import glob
import cv2
import json
import datetime
import re
import string
import copy
import nltk
import itertools
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import tensorflow as tf
from gensim.models import word2vec
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, Lambda, Input, Bidirectional,  Add, CuDNNGRU, Concatenate, Dropout, GRU, LSTM, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Conv1D, AveragePooling2D, MaxPooling2D, MaxPooling1D, GlobalMaxPooling2D
from keras.models import Model, load_model, Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import layer_utils, plot_model
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.applications import InceptionV3
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K
from keras.initializers import glorot_uniform
from __future__ import print_function
import keras
from keras import backend as k
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import skimage
import six
import scipy

In [0]:
ls

In [0]:
time_t = datetime.datetime.utcnow()
dataset=[]
with open('shared_task_test.jsonl', 'r') as f:
  dataset = f.read().split('\n')
  dataset.pop()

for i in range(len(dataset)):
  dataset[i] = json.loads(dataset[i])
  
X_test_claim = []
for i in dataset:
  X_test_claim.append(i['claim'])
  #X_train_label.append(i['label'])
del dataset
print(datetime.datetime.utcnow() - time_t)

0:00:00.121351


### Text Tokenizer

In [0]:
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
def getWikiPage(i):
  path = '../wiki-pages/wiki-'
  content=[]
  with open(path+'%03d'%i+'.jsonl', 'r') as f:
    document = f.read().split('\n')
    document.pop()
    for j in range(len(document)):
      document[j] = json.loads(document[j])
    lines=[]
    idd=[]
    line_no=[]
    for j in document:      
      temp = j['lines']
      temp = temp.split('\n')
      temp.pop()
      for i in range(len(temp)):
        if len(temp[i]) < 35 :
          continue
        lines.append(temp[i])
        idd.append(j['id'])
        line_no.append(i)
  return lines, idd, line_no

def printMaxLen():
  path = 'wiki-pages/wiki-'
  minn = 10000000
  maxx = 0
  for i in range(1,110):
    cnt = 0
    with open(path+'%03d'%i+'.jsonl', 'r') as f:
      document = f.read().split('\n')
      document.pop()
      for j in range(len(document)):
        document[j] = json.loads(document[j])
      for j in document:
        temp = j['text']
        temp = sent_tokenize(temp)
        for sent in temp:
          if len(sent)>40:
            cnt += 1
    maxx = max(cnt, maxx)
    minn = min(cnt, minn)
  print(maxx, minn)
  return

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#printMaxLen()
#print(preprocess(getWikiPage(3))[0:100])
#getWikiPage(7)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tag import pos_tag
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

stop = list(set(stopwords.words('english')))
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def strip_accents(text):
    """
    Strip accents from input String.

    :param text: The input string.
    :type text: String.

    :returns: The processed String.
    :rtype: String.
    """
    try:
        text = unicode(text, 'utf-8')
    except (TypeError, NameError): # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
      stemmed.append(stemmer.stem(item))
    return stemmed
def lemmatize_tokens(tokens, lemmatizer):
    lemmatized=[]
    for word, tag in pos_tag(tokens):
      temp = '0'
      if tag.startswith("NN"):
          temp =  lemmatizer.lemmatize(word, pos='n')
      elif tag.startswith('VB'):
          temp = lemmatizer.lemmatize(word, pos='v')
      elif tag.startswith('JJ'):
          temp = lemmatizer.lemmatize(word, pos='a')
      else:
          temp = word
      lemmatized.append(temp)
    return lemmatized

def tokenize(text):
    b = ['-lrb-', '-rrb-', '-rsb-', '-lsb-', '-lcb-', '-rcb-']
    text = text.lower()
    text = re.sub("\'"+'[A-Za-z]*', '', text)
    text = strip_accents(text)
    tokens = nltk.word_tokenize(text)
    tokens = [token for token in tokens if token not in string.punctuation and token not in b and token not in stop and len(token)>2 and not token.isdigit()]
    stems = lemmatize_tokens(tokens, lemmatizer)
    return stems
  
def preprocess(listt):
    new_list=[]
    for i in listt:
      z = tokenize(i)
      z = ' '.join(x for x in z)
      new_list.append(z)
    #print('Preprocess done.')
    return new_list 
  
def pad_sentence(listt, length):
    new_list = []
    max = 0
    for string in listt:
      if len(string)>max:
        max = len(string)
     
    if length < max:
      max = length
    print("Maximum sequence length: "+str(max))
    for string in listt:
      z = max - len(string)
      if z > 0:
        temp = string + ['<PAD/>']*z
      else:
        temp = string[0:max]
      new_list.append(temp)
    return new_list
  
def build_vocab(sentences):
    word_counts = Counter(itertools.chain(*sentences))
    vocabulary_inv = [x[0] for x in word_counts.most_common()]
    vocabulary = {x: i for i, x in enumerate(vocabulary_inv)}
    print("Length of vocabulary: " + str(len(vocabulary)))
    return [vocabulary, vocabulary_inv]  
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Mounting with Google Drive for getting pretrained model

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
#!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
#!apt-get update -qq 2>&1 > /dev/null
#!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
import time
import sys
import re
from subprocess import call
import numpy as np
from nltk import TweetTokenizer
from nltk.tokenize.stanford import StanfordTokenizer

In [0]:
cd fastText-0.1.0

[Errno 2] No such file or directory: 'data/fastText-0.1.0'
/content/data/fastText-0.1.0


In [0]:
!cp ../drive/colab/wiki_unigrams.bin model.bin

### Loading the pretrained model

In [0]:
MODEL_WIKI_UNIGRAMS = 'model.bin'
SNLP_TAGGER_JAR = 'stanford-postagger-full-2018-02-27/stanford-postagger.jar' 
FASTTEXT_EXEC_PATH = './fasttext'

def get_embeddings_for_preprocessed_sentences(sentences, model_path, fasttext_exec_path):
    """Arguments:
        - sentences: a list of preprocessed sentences
        - model_path: a path to the sent2vec .bin model
        - fasttext_exec_path: a path to the fasttext executable
    """
    timestamp = str(time.time())
    test_path = '_fasttext.txt'
    embeddings_path = '_fasttext_embeddings.txt'
    dump_text_to_disk(test_path, sentences)
    call(fasttext_exec_path+
          ' print-sentence-vectors '+
          model_path + ' < '+
          test_path + ' > ' +
          embeddings_path, shell=True)
    embeddings = read_embeddings(embeddings_path)
    #os.remove(test_path)
    #os.remove(embeddings_path)
    assert(len(sentences) == len(embeddings))
    return np.array(embeddings)
    new = np.zeros((len(embeddings), 600))
    
    for i in range(len(embeddings)):
      r = len(embeddings[i])
      for j in range(r):
        new[i][j] = embeddings[i][j]
    return new

def read_embeddings(embeddings_path):
    """Arguments:
        - embeddings_path: path to the embeddings
    """
    with open(embeddings_path, 'r') as in_stream:
        embeddings = []
        #print(in_stream.read())
        for line in in_stream:          
            line = line.strip()
            #line = re.sub('[A-Za-z]+ ', '', line)
            line = line.split('  ')
            line = line[1].replace(' ',',')[1:]
            line = '['+line+']'
            embeddings.append(eval(line))
        return embeddings

def dump_text_to_disk(file_path, X, Y=None):
    """Arguments:
        - file_path: where to dump the data
        - X: list of sentences to dump
        - Y: labels, if any
    """
    with open(file_path, 'w') as out_stream:
        if Y is not None:
            for x, y in zip(X, Y):
                out_stream.write('__label__'+str(y)+' '+x+' \n')
        else:
            for x in X:
                #print(x)
                out_stream.write(x+' \n')

def get_sentence_embeddings(sentences, ngram='bigrams', model='concat_wiki_twitter'):
    """ Returns a numpy matrix of embeddings for one of the published models. It
    handles tokenization and can be given raw sentences.
    Arguments:
        - ngram: 'unigrams' or 'bigrams'
        - model: 'wiki', 'twitter', or 'concat_wiki_twitter'
        - sentences: a list of raw sentences ['Once upon a time', 'This is another sentence.', ...]
    """
    wiki_embeddings = None
    twitter_embbedings = None
    tokenized_sentences_NLTK_tweets = None
    tokenized_sentences_SNLP = None
    if model == "wiki" or model == 'concat_wiki_twitter':
        #tknzr = StanfordTokenizer(SNLP_TAGGER_JAR, encoding='utf-8')
        #s = ' <delimiter> '.join(sentences) #just a trick to make things faster
        #tokenized_sentences_SNLP = tokenize_sentences(tknzr, [s])
        tokenized_sentences_SNLP = preprocess(sentences)
        assert(len(tokenized_sentences_SNLP) == len(sentences))
        if ngram == 'unigrams':
            wiki_embeddings = get_embeddings_for_preprocessed_sentences(tokenized_sentences_SNLP, \
                                     MODEL_WIKI_UNIGRAMS, FASTTEXT_EXEC_PATH)
        else:
            wiki_embeddings = get_embeddings_for_preprocessed_sentences(tokenized_sentences_SNLP, \
                                     MODEL_WIKI_BIGRAMS, FASTTEXT_EXEC_PATH)
    if model == "twitter" or model == 'concat_wiki_twitter':
        tknzr = TweetTokenizer()
        tokenized_sentences_NLTK_tweets = tokenize_sentences(tknzr, sentences)
        if ngram == 'unigrams':
            twitter_embbedings = get_embeddings_for_preprocessed_sentences(tokenized_sentences_NLTK_tweets, \
                                     MODEL_TWITTER_UNIGRAMS, FASTTEXT_EXEC_PATH)
        else:
            twitter_embbedings = get_embeddings_for_preprocessed_sentences(tokenized_sentences_NLTK_tweets, \
                                     MODEL_TWITTER_BIGRAMS, FASTTEXT_EXEC_PATH)
    if model == "twitter":
        return twitter_embbedings
    elif model == "wiki":
        return np.array(wiki_embeddings)
    elif model == "concat_wiki_twitter":
        return np.concatenate((wiki_embeddings, twitter_embbedings), axis=1)
    sys.exit(-1)

In [0]:
#printm()
import sent2vec
model = sent2vec.Sent2vecModel()
model.load_model('model.bin')
#printm()

In [0]:
doc1=["Today is a hot day. I don't feel like going out because i am gonna get sick. so i better stay at home", "The franchise began as a pair of video games for the original Game Boy that were developed by Game Freak and published by Nintendo. It spans a video game series, a trading card game, an anime television series, a film series, books, manga comics, music, toys, and merchandise. Pokémon is the highest-grossing media franchise of all time,[9][10] with over ¥6 trillion[11] ($59.1 billion) in total revenue.", "Football is a family of team sports that involve, to varying degrees, kicking a ball with a foot to score a goal. Unqualified, the word football is understood to refer to whichever form of football is the most popular in the regional context in which the word appears.", "There are a number of references to traditional, ancient, or prehistoric ball games played by indigenous peoples in many different parts of the world.[3][4][5] Contemporary codes of football can be traced back to the codification of these games at English public schools during the nineteenth century."]
test = ["pikachu is my most fav animal"]

my_embeddings = model.embed_sentences(test+doc1)
print('got')
print(my_embeddings.shape)
print(my_embeddings[0].shape, my_embeddings[1:].shape)
sim = cosine_similarity(my_embeddings[0:2], my_embeddings[1:])
print(sim.shape)
print(doc1[np.argmax(sim[0])])

In [0]:
import csv
time_t = datetime.datetime.utcnow()
def getLiarData(path):
  X=[]
  dic={}
  with open('../liar_evidence.tsv','r', encoding='cp850', errors='replace') as f:
    reader = csv.reader(f, delimiter='\t')
    
    for row in reader:
      dic[int(row[0])]=row[1]
      
  with open('../'+path+'.tsv', 'r', encoding='cp850', errors='replace') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
      temp={}
      temp['id'] = int(row[0][:-5])
      if temp['id'] not in dic.keys():
        continue
      label = 0
      strr = row[1].lower()
      if strr == 'true':
        label=0
      elif strr == 'mostly-true':
        label=1
      elif strr == 'half-true':
        label=2
      elif strr == 'barely-true':
        label=3
      elif strr == 'false':
        label=4
      else:
        label=5
      temp['label'] = label
      temp['claim'] = row[2]
      
      evidences = nltk.sent_tokenize(dic[temp['id']])
      claim_embedding = model.embed_sentence(preprocess([temp['claim']])[0])
      ev_embeddings = model.embed_sentences(preprocess(evidences))
      
      sim_array = cosine_similarity(claim_embedding, ev_embeddings)
      idx = np.argmax(sim_array)
      temp['evidence'] = evidences[idx]
      X.append(temp)
    print(len(X))
    print(datetime.datetime.utcnow() - time_t) 
  return X
   

In [0]:
X=getLiarData('valid')

with open('liar_valid_train.json', 'w') as output:
  json.dump(X, output)
  
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

drive_service = build('drive', 'v3')
file_metadata = {
  'name': 'liar_valid_train.json',
  'mimeType': 'application/json'
}
media = MediaFileUpload('liar_valid_train.json', 
                        mimetype='application/json',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id'))) 

1284
0:28:50.495775
File ID: 182BcM_ErQthPpSLhjyC40YhcZAbYQHL2


## Conversion of Sentences to vectors and calculating Cosine Similarity

In [0]:
time_t = datetime.datetime.utcnow()
'''
X_claim = np.load('X_claim.npy')
X_support = np.load('X_support.npy')
X_sim = np.load('X_sim.npy')
'''
X_claim = preprocess(X_test_claim)
X_support = np.ndarray(shape=(len(X_claim), 5), dtype=object)
X_id = np.ndarray(shape=(len(X_claim), 5), dtype=object)
X_sim = np.zeros((len(X_claim), 5))
X_line_no = np.zeros((len(X_claim), 5))
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), strip_accents='unicode')
tfidf_matrix = tfidf_vectorizer.fit(X_claim)
#claim_embeddings = model.embed_sentences(preprocess(X_claim))
#print(claim_embeddings.shape)
for i in range(1,35): 
  time_tt = datetime.datetime.utcnow()
  X_doc1, X_doc_id, X_doc_line = getWikiPage(i)
  l = int(len(X_doc1)/7)
  X_doc = preprocess(X_doc1)
  tfidf_matrix = tfidf_vectorizer.transform(X_claim + X_doc)
  print(tfidf_matrix.shape)
  
  claim_tfidf = tfidf_matrix[0:len(X_claim)]
  ev_tfidf = tfidf_matrix[len(X_claim):]
   
  for k in range(1,8):
    ll = l*k
    if k==7:
      ll = len(X_doc)
    sim_array = cosine_similarity(claim_tfidf, ev_tfidf[l*(k-1):ll]) 
    print(sim_array.shape)
    for j in range(len(X_claim)):
      temp_sim = np.max(sim_array[j])
      temp_support = X_doc[l*(k-1)+np.argmax(sim_array[j])]
      temp_id = X_doc_id[l*(k-1)+np.argmax(sim_array[j])]
      temp_line_no = X_doc_line[l*(k-1)+np.argmax(sim_array[j])]
      for m in range(5):
        if temp_sim > X_sim[j][m]:
          X_sim[j][m], temp_sim = temp_sim, X_sim[j][m]
          X_support[j][m], temp_support = temp_support, X_support[j][m]
          X_id[j][m], temp_id = temp_id, X_id[j][m]
          X_line_no[j][m], temp_line_no = temp_line_no, X_line_no[j][m]
          
    del sim_array
    
  #np.save('X_claim.npy', X_claim)
  #np.save('X_support.npy', X_support)
  #np.save('X_sim.npy', X_sim)
  
  print('wiki-page: '+str(i)+ ' time taken: '+str(datetime.datetime.utcnow() - time_tt))
  #del tfidf_matrix 
print(datetime.datetime.utcnow() - time_t) 
 

X=[]
for i in range(len(X_claim)):
  temp={}
  temp['claim'] = X_claim[i]
  temp['support'] = X_support[i]
  temp['sim'] = X_sim[i]
  temp['id'] = X_id[i]
  temp['line_no'] = X_line_no[i]
  X.append(temp)
print('size of test: '+str(len(X)))
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

dumped = json.dumps(X, cls=NumpyEncoder)

with open('notun_test_tfidf.json', 'w') as output:
  json.dump(dumped, output)
  
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

drive_service = build('drive', 'v3')
file_metadata = {
  'name': 'notun_test_tfidf.json',
  'mimeType': 'application/json'
}
media = MediaFileUpload('notun_test_tfidf.json', 
                        mimetype='application/json',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id'))) 

In [0]:

X=[]
for i in range(len(X_claim)):
  temp={}
  temp['claim'] = X_claim[i]
  temp['support'] = X_support[i]
  temp['sim'] = X_sim[i]
  temp['id'] = X_id[i]
  temp['line_no'] = X_line_no[i]
  X.append(temp)
print('size of test: '+str(len(X)))
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

dumped = json.dumps(X, cls=NumpyEncoder)

with open('notun_test_tfidf.json', 'w') as output:
  json.dump(dumped, output)
  
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

drive_service = build('drive', 'v3')
file_metadata = {
  'name': 'notun_test_tfidf.json',
  'mimeType': 'application/json'
}
media = MediaFileUpload('notun_test_tfidf.json', 
                        mimetype='application/json',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id'))) 